In [2]:
import pandas as pd

In [10]:
lecture_df = pd.read_csv("data/lectures.csv")
question_df = pd.read_csv("data/questions.csv")
response_df = pd.read_csv("data/responses.csv")
train_df = pd.read_csv("data/short_train.csv")

train_df.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [5]:
train_test_df = train_df['content_type_id'] == 0
train_test_df.head()

0    True
1    True
2    True
3    True
4    True
Name: content_type_id, dtype: bool

In [18]:
train_df_0 = train_df[train_test_df]
train_df_0.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [19]:
item_counts = train_df.groupby('user_id').count()
print(item_counts)

         row_id  timestamp  content_id  content_type_id  task_container_id  \
user_id                                                                      
115          46         46          46               46                 46   
124          30         30          30               30                 30   
2746         20         20          20               20                 20   
5382        128        128         128              128                128   
8623        112        112         112              112                112   
...         ...        ...         ...              ...                ...   
6799950      30         30          30               30                 30   
6802969      13         13          13               13                 13   
6804274     322        322         322              322                322   
6811009      40         40          40               40                 40   
6812883    2059       2059        2059             2059         

In [14]:
train_df.count()

row_id                            349999
timestamp                         349999
user_id                           349999
content_id                        349999
content_type_id                   349999
task_container_id                 349999
user_answer                       349999
answered_correctly                349999
prior_question_elapsed_time       342085
prior_question_had_explanation    348786
dtype: int64

In [15]:
len(train_df)

349999

In [20]:
len(train_df_0)

343298